In [20]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from scipy.spatial import HalfspaceIntersection, ConvexHull
from itertools import combinations
import networkx as nx

#### The Lemke-Howson algorithm, a deeper look

Here I had a lot of trouble understanding the algorithm, but found another great resource at https://cnl.gmu.edu/TAVRI/research/LemkeHowson.pdf, which was more intuitive, and what I follow for this section.

Here $U_1$ and $U_2$ are $(n,m)$ matricies representing the payoffs of both players. $s_1$ and $s_2$ are the strategies (probability vectors). $v_1$ and $v_2$ are constants representing the best each player can do, given the other player's strategy. Finally, $r_1$ and $r_2$ are vectors representing the gap between the reward for each action and the best action, given the other player's strategy. 

At equilibrium we have:

$$
\begin{align*}
\text{} \quad U_1\cdot s_2 + r_1 & = v_1 \\
\text{} \quad U_2^T\cdot s_1 + r_2 & = v_2 \\
\text{} \quad s_1\cdot r_1 & = 0 \\
\text{} \quad s_2\cdot r_2 & = 0 \\
\text{} \quad \sum s_1 & = 1\\
\text{} \quad \sum s_2 & = 1
\end{align*}
$$

The first two equations state what we had above, that the benefit of each action plus some number equals the benefit of the best action. The third and fourth lines state that an action either has 0 probability, or it is the best action (or tied-best). The fifth and sixth state that the strategies are probabilities. We also need the condition that all variables $\geq 0$, but I have left that out for now. 

We can simplify this further by dividing the first two lines by $v_1$ and $v_2$ respectively:

$$
\begin{align*}
\text{} \quad U_1\cdot s_2/v_1 + r_1/v_1 & = 1 \\
\text{} \quad U_2^T\cdot s_1/v_2 + r_2/v_2 & = 1 \\
\text{} \quad s_1\cdot r_1 & = 0 \\
\text{} \quad s_2\cdot r_2 & = 0 \\
\text{} \quad \sum s_1 & = 1\\
\text{} \quad \sum s_2 & = 1 \\
\end{align*}
$$

Defining $s_1^\prime=s_1/v_2$, $s_2^\prime=s_2/v_1$, $r_1^\prime=r_1/v_1$, and $r_2^\prime=r_2/v_2$ we can rewrite the equations above and get:

$$
\begin{align*}
\text{} \quad U_1\cdot s_2^\prime + r_1^\prime & = 1 \\
\text{} \quad U_2^T\cdot s_1^\prime + r_2^\prime & = 1 \\
\text{} \quad (v_2s_1^\prime)\cdot (v_1r_1^\prime) & = 0 \\
\text{} \quad (v_1s_2^\prime)\cdot (v_2r_2^\prime) & = 0 \\
\text{} \quad \sum s_1^\prime & = 1/v_2 \\
\text{} \quad \sum s_2^\prime & = 1/v_1 \\
\end{align*}
$$

In the third and fourth lines the constants don't matter, so we can remove them (we also do this for the condition that all variables are $\geq 0$):

$$
\begin{align*}
\text{} \quad U_1\cdot s_2^\prime + r_1^\prime & = 1 \\
\text{} \quad U_2^T\cdot s_1^\prime + r_2^\prime & = 1 \\
\text{} \quad s_1^\prime\cdot r_1^\prime & = 0 \\
\text{} \quad s_2^\prime\cdot r_2^\prime & = 0 \\
\text{} \quad \sum s_1^\prime & = 1/v_2 \\
\text{} \quad \sum s_2^\prime & = 1/v_1 \\
\end{align*}
$$

We can remove the last two equations as the utility (and equilibrium) don't depend on any scaling of the utility function. I.e., say $v_1=c_1v_1^\text{truth}$. As we are allowed any $c_1$ we can just use $c_1=1/(v_1^\text{truth}\sum s_2^\prime)$ to get $v_1=1/(\sum s_2^\prime)$. If we plug this back in the last two equations can be removed. Essentially, we are just agreeing to normalise $s_1^\prime$ and $s_2^\prime$ when outputing the actual strategies.

$$
\begin{align*}
\text{} \quad U_1\cdot s_2^\prime + r_1^\prime & = 1 \\
\text{} \quad U_2^T\cdot s_1^\prime + r_2^\prime & = 1 \\
\text{} \quad s_1^\prime\cdot r_1^\prime & = 0 \\
\text{} \quad s_2^\prime\cdot r_2^\prime & = 0 \\
\end{align*}
$$

The first equation above describes $n$ different equality relationships, in $n+m$ variables. The second equation above describes $m$ different equality relationships, also in a set of $n+m$ other variables. We know in order to solve the variables in the first equation we have to set $m$ of the variables to 0, and likewise we need to set $n$ of the variables in the second to 0 to solve them. Equivalently, we need to choose $n$ variables in the first to be non-zero, and $m$ in the second to be non-zero. All that remains after is to test the final 2 equations are also true for the result.

Lets take this example:

$
\begin{array}{c|cc}
\text{} & \text{X} & \text{Y} \\
\hline
\text{X} & 3,1 & 1,0 \\
\text{Y} & 0,0 & 2,2 \\
\end{array}
$

Below we iterate through all combinations of $n$ and $m$ non-zero variables in the first and second equations. Note, that the first equation is encoded as 
$\big[I\text{ }U_1\big] \big(\begin{array}{c}r_1 \\s_2 \\\end{array}\big)$ and likewise $\big[U_2\text{ }I\big] \big(\begin{array}{c}s_1 \\r_2 \\\end{array}\big)$ for the second (This way the variables to be calculated are in order and correspond to one another). As a final step we check that the last 2 equations are met by the solution.

In [43]:
U1_I = np.array([
    [1,0,3,1],
    [0,1,0,2]
])
U2T_I = np.array([
    [1,0,1,0],
    [0,2,0,1]
])
for comb1 in list(combinations(range(4), 2)):
    for comb2 in list(combinations(range(4), 2)):
        if not np.linalg.det(U1_I[:,comb1])==0 and not np.linalg.det(U2T_I[:,comb2])==0: # check there is a solution
            solu1 = np.zeros(4)
            solu2 = np.zeros(4)
            solu1[np.array(comb1)] = np.linalg.inv(U1_I[:,comb1]).dot(np.ones(2))
            solu2[np.array(comb2)] = np.linalg.inv(U2T_I[:,comb2]).dot(np.ones(2))
            if all(solu1>=0) and all(solu2>=0): # check variables >0
                r1 = solu1[:2]
                s2 = solu1[2:]
                s1 = solu2[:2]
                r2 = solu2[2:]
                if np.sum(s1>0) and np.sum(s2>0): # need to just check the solution (0,0) wasn't found.
                    if s1.dot(r1)==0 and s2.dot(r2)==0:
                        s1 = np.round(s1/np.sum(s1),2)
                        s2 = np.round(s2/np.sum(s2),2)
                        print(f"solution: s1 {s1}, s2 {s2}")

solution: s1 [0. 1.], s2 [0. 1.]
solution: s1 [1. 0.], s2 [1. 0.]
solution: s1 [0.67 0.33], s2 [0.25 0.75]


This works and will find all nash equilibria. 

Note: to meet the bottom two equations all we need is that the $n$ non-zero entries in $\big(\begin{array}{c}r_1 \\s_2 \\\end{array}\big)$ are different indexes than the $m$ entries in $\big(\begin{array}{c}s_1 \\r_2 \\\end{array}\big)$. This corresponds to choosing different sets of columns from $\big[I\text{ }U_1\big]$ and $\big[U_2\text{ }I\big]$.

For example, the solution $s_1=[0,1]$,$s_2=[1,0]$ corresponds to the following choice (non-zero values in red, power is the action taken):

$
\big[I\text{ }U_1\big]=
\begin{array}{cccc}
\textcolor{red}{r_1^1} & r_1^2 & \textcolor{red}{s_2^1} & s_2^2 \\
\hline
1 & 0 & 3 & 1 \\
0 & 1 & 0 & 2 \\
\end{array}
\quad
\big[U_2\text{ }I\big]=
\begin{array}{cccc}
s_1^1 & \textcolor{red}{s_1^2} & r_2^1 & \textcolor{red}{r_2^2} \\
\hline
1 & 0 & 1 & 0 \\
0 & 2 & 0 & 1 \\
\end{array}
$

So, we can find solutions this way. But we can also speed this up a lot, by turning it into a path-finding algorithm. 

Say that we have a single column in $\big[U_2\text{ }I\big]$ which is also in $\big[I\text{ }U_1\big]$. E.g., $s_1^2=r_1^2$. By setting that column to 0 we can remove the conflict, introducing at most 1 new conflict for another column. Switching between tables, we then draw a path to a solution.

**Example**

Consider the above problem again, starting out at the (invalid) point where $s_1$ and $s_2$ are 0:

$
\big[I\text{ }U_1\big]=
\begin{array}{cccc}
\textcolor{red}{r_1^1} & \textcolor{red}{r_1^2} & s_2^1 & s_2^2 \\
\hline
1 & 0 & 3 & 1 \\
0 & 1 & 0 & 2 \\
\end{array}
\quad
\big[U_2\text{ }I\big]=
\begin{array}{cccc}
s_1^1 & s_1^2 & \textcolor{red}{r_2^1} & \textcolor{red}{r_2^2} \\
\hline
1 & 0 & 1 & 0 \\
0 & 2 & 0 & 1 \\
\end{array}
$

This fits the criteria, but the probabilities are 0. We choose an arbitrary new column index in the first table ($s_2^1$) and set the old column ($r_1^1$) to 0. This creates a clash (between $s_2^1$ and $r_2^1$):

$
\big[I\text{ }U_1\big]=
\begin{array}{cccc}
r_1^1 & \textcolor{red}{r_1^2} & \textcolor{red}{s_2^1} & s_2^2 \\
\hline
1 & 0 & 3 & 1 \\
0 & 1 & 0 & 2 \\
\end{array}
\quad
\big[U_2\text{ }I\big]=
\begin{array}{cccc}
s_1^1 & s_1^2 & \textcolor{red}{r_2^1} & \textcolor{red}{r_2^2} \\
\hline
1 & 0 & 1 & 0 \\
0 & 2 & 0 & 1 \\
\end{array}
$

To fix the clash in the second we remove the problematic column in the second, and choose a new one:

$
\big[I\text{ }U_1\big]=
\begin{array}{cccc}
r_1^1 & \textcolor{red}{r_1^2} & \textcolor{red}{s_2^1} & s_2^2 \\
\hline
1 & 0 & 3 & 1 \\
0 & 1 & 0 & 2 \\
\end{array}
\quad
\big[U_2\text{ }I\big]=
\begin{array}{cccc}
\textcolor{red}{s_1^1} & s_1^2 & r_2^1 & \textcolor{red}{r_2^2} \\
\hline
1 & 0 & 1 & 0 \\
0 & 2 & 0 & 1 \\
\end{array}
$

And voila! We have a solution. This corresponds to $s_1=[1,0]$, $s_2=[0,1]$.

